# Instalasi Paket

In [84]:
#!pip install tweepy
#!pip install textblob

# Download Punkt

In [85]:
#import nltk
#nltk.download('punkt')

# Ambil Twit

In [2]:
import tweepy

In [29]:
topic = 'mudik'

In [27]:
consumer_key  = 'API Key dari Twitter'
consumer_secret = 'API secret key'
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth)

In [ ]:
data = []
for tweet in tweepy.Cursor(api.search, q=topic).items(200):
    status = api.get_status(tweet.id, tweet_mode='extended')
    try:
        text = status.retweeted_status.full_text
    except AttributeError:
        text = status.full_text
    data.append([text, 1])

# Simpan sebagai csv

In [3]:
import csv

In [ ]:
with open('tweet.csv', 'w') as csvfile:
    wr = csv.writer(csvfile)
    for d in data:
        wr.writerow(d)

# Ambil twit yang sudah diberi label

In [4]:
labeled_tweet = []
with open('labeled.csv', 'r') as csvfile:
    cf = csv.reader(csvfile)
    for row in cf:
        labeled_tweet.append(row)

# Pisahkan twit positif dan negatif

In [5]:
pos = []
neg = []

for l in labeled_tweet:
    if l[1] == '1':
        pos.append((l[0], 1))
    else:
        neg.append((l[0], 0))

# Banyak masing-masing data

In [6]:
len(pos), len(neg)

(98, 102)

# Ambil masing-masing opini sebanyak 75 sebagai data train dan 20 sebagai data test

In [7]:
pos_train = pos[:75]
pos_test = pos[-20:]
neg_train = neg[:75]
neg_test = neg[-20:]

In [8]:
len(pos_train), len(pos_test), len(neg_train), len(neg_test)

(75, 20, 75, 20)

# Gabung data train dan data test

In [9]:
train = pos_train + neg_train
test = pos_test + neg_train

# Buat Model

In [10]:
from textblob.classifiers import NaiveBayesClassifier

In [11]:
cl = NaiveBayesClassifier(train)

# Uji Model pada data test

In [12]:
for t in test:
    print(t[0], '|------>', cl.classify(t[0]))

Banyak yg dri kmren nafsu ingin bully Jokowi karena perkataan soal Mudik dan pulang kampung.

Tapi sayang, setelah mereka buka KBBI dan diperkuat oleh penjelasan ahli linguistik (Guru besar UI), secara otomatis bullian mereka telah menampar muka sendiri. https://t.co/dlm9RAJmUw |------> 1
Pulkam itu sifatnya parsial, dilakukan dlm waktu berbeda, jadi pemantauannya lbh mudah di msg2 daerah asal.

Mudik itu sifatnya simultan, dilakukan dlm waktu yg sama dan biasanya dlm jumlah besar. Pemda sulit memantau di daerah asal.

Sederhana gini sebenarnya, apa yg diributkan? |------> 1
@Janfri_75 @CommuterLine KAJJ kan udh mulai gaboleh beroperasi karna gaboleh mudik. tmbahin keberangkatannya dong min untuk jalur biru (Bekasi/Cikarang) biar bisa lebih maksimal min.. |------> 0
@adekurniawaw Bang aji lagi mudik atau pulang kampung? |------> 0
Dari Jokowi soal mudik dan kampung halaman ---- [ ucapan prabowo soal saksi jokowi + penangkapan ravio patra ] 

Ada rangkaian upaya tuk menghambat arus info

# Akurasi Model yang dibuat

In [13]:
cl.accuracy(test)

0.8736842105263158

# Uji pada data live

In [34]:
for tweet in tweepy.Cursor(api.search, q=topic).items(10):
    status = api.get_status(tweet.id, tweet_mode='extended')
    try:
        text = status.retweeted_status.full_text
    except AttributeError:
        text = status.full_text
        
    print(text, '|----->', cl.classify(text))

Clear ya.., yang bicara Prof. Dr. Rahayu Surtiati Hidayat, ahli bahasa dari Universitas Indonesia. Bukan kaum kaleng-kalengan dan recehan..🍮

https://t.co/zHaO7aL7rG |-----> 0
Gaboleh mudik tapi pulang kampung boleh |-----> 0
Penerbangan Mudik di Zona PSBB-Merah Dilarang, Kualanamu Tetap Beroperasi https://t.co/lKSMEKyENo https://t.co/2rHO7e8J5h |-----> 0
7.Artikel ini ringan dan mungkin tidak penting. Namun siapa tahu ada kadrun yang gagal paham dan membully Presiden Jokowi karena istilah mudik dan pulang kampung. Jadi anda bisa dengan mudah bisa mengcounternya. |-----> 1
Masih belum paham beda mudik dan pulang kampung? Ahli bahasa dari Universitas Indonesia memberikan penjelasan soal pernyataan Presiden Jokowi yang viral itu. https://t.co/jEZpR4Rc6Y |-----> 1
Kementerian Perhubungan menghentikan seluruh operasional kapal laut yang mengangkut penumpang mulai 24 April 2020 hingga 8 Juni 2020 selama aturan larangan mudik diberlakukan. 

LIVE !! Direktur Lalu Lintas dan Angkutan Laut Dir